In [1]:
import casadi as ca
import numpy as np
import ciropt as co
import sympy as sp

In [2]:
L_smooth = 1.
mu = 0.0001
Capacitance = 2.
Inductance = 2.
R = 1.

# solver = "ca"
solver = "cvx_fix_sdp"

In [3]:
# num_fi = 1

# problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)

# params = {'b': 2.668461572568973,
#             'h': 6.0353467370354705,
#             'd': 0.7721221265094984,
#             'alpha': 0.4915886656470814,
#             'beta': 0.44375266078718323}

In [7]:
L_smooth = 1.
mu = 0.001
Inductance = 5.
R = 1.

num_fi = 3
problem = co.admm_consensus(num_fi, mu, L_smooth, R, Inductance)

In [8]:
bounds = {  'b': {"ub": 0.000001, "lb": 0.000001},
            'h': {"ub": 0.000001, "lb": 0.000001}}

vars, sol, sp_exp = problem.solve(solver="ca", verbose=True, debug=True, bounds=bounds)

dim_G=14, dim_F=12
opts={'ipopt.max_iter': 50000}
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    12458
Number of nonzeros in inequality constraint Jacobian.:      126
Number of nonzeros in Lagrangian Hessian.............:      652

Total number of variables............................:      340
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      240
Total number of inequality constraints...............:      126
        inequality constraints with only lower bounds:      124
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -0.0000000e+00 9.99e-0

ValueError: too many values to unpack (expected 3)

In [ ]:
params = {'b': -6.586851988799446e-12,
 'h': 2.0050490196829352e-11,
 'd': 0,
 'alpha': 0.00021484442616463077,
 'beta': 0.18371375512916682}


params = {  'b': 0.000001,
            'h': 0.000001,
            'd': 0.,
            'alpha': 0.5,
            'beta': 0.5}

bounds = {  'b': {"ub": 0.000001, "lb": 0.000001},
            'h': {"ub": 0.000001, "lb": 0.000001}}

In [ ]:
# vars, prob, sp_exp = problem.solve(solver=solver, verbose=True, debug=True)
vars, prob, sp_exp = problem.solve(solver=solver, verbose=False, debug=True, params=params)
print(prob.status)

In [ ]:
init_vals = {}
Fi_size = vars["lambda"].size // num_fi
size_I = int(np.ceil(np.sqrt(Fi_size)))
for f_idx in range(num_fi):
    lamb_i = np.zeros((size_I, size_I))
    vec = np.maximum(vars["lambda"][f_idx * Fi_size : (f_idx+1) * Fi_size], 0)
    count  = 0
    for i in range(size_I):
        for j in range(size_I):
            if i == j: continue
            lamb_i[i, j] = vec[count]
            count += 1
    assert count == size_I * (size_I - 1)
    init_vals["lamb%d"%f_idx] = lamb_i

Z = vars["Z"]
Lamb, V = np.linalg.eigh(Z)
assert np.allclose(V @ np.diag(Lamb) @ V.T, Z)
Z_plus = V @ np.diag(np.maximum(Lamb, 0)) @ V.T
P = np.linalg.cholesky(Z_plus)

init_vals["P_full"] = P

In [ ]:
# problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)

In [ ]:
problem = co.admm_consensus(num_fi, mu, L_smooth, R, Inductance)

In [ ]:
vars, sol, sp_exp = problem.solve(solver="casadi_short", verbose=True, debug=True, bounds=bounds)#init_vals=init_vals)

dim_G=14, dim_F=12


ValueError: too many values to unpack (expected 3)

In [ ]:
print(problem.opti.stats()["iter_count"])

In [ ]:
vars